In [ ]:
!pip install umap-learn
!pip install hdbscan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: cython
    Found existing installation: Cython 3.0.10
    Uninstalling Cython-3.0.10:
      Successfully uninstalled Cython-3.0.10


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
#import umap.umap_ as umap
#import hdbscan
from transformers import AutoModel, AutoTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
scibert_model = 'allenai/scibert_scivocab_uncased'
tokenizer = AutoTokenizer.from_pretrained(scibert_model)
model = AutoModel.from_pretrained(scibert_model)
model = model.eval().to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
texts  = [
    "function selectActivities(start, finish, N):",
    "    activities = []",
    "    for i from 0 to N-1:",
    "        activities.append((start[i], finish[i], i))",
    "    sort(activities, by finish time)",
    "    selectedActivities = []",
    "    selectedActivities.append(activities[0])",
    "    lastFinishTime = activities[0].finish",
    "    for i from 1 to N-1:",
    "        if activities[i].start >= lastFinishTime:",
    "            selectedActivities.append(activities[i])",
    "            lastFinishTime = activities[i].finish",
    '    print "Number of activities selected:", length(selectedActivities)',
    '    print "Selected activities:"',
    "    for each activity in selectedActivities:",
    '        print activity.index'
]


In [ ]:
tokenized_texts = tokenizer(texts, padding=True, truncation=True, return_tensors='pt').to(device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
with torch.no_grad():
    outputs = model(**tokenized_texts)
hidden_states = outputs.last_hidden_state

In [ ]:
sentence_embeddings = hidden_states.mean(dim=1).cpu().numpy()

In [ ]:
from sklearn.cluster import KMeans
n_clusters = 5 # Desired number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(sentence_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
print("SciBERT Cluster labels:", cluster_labels)

SciBERT Cluster labels: [1 3 0 1 1 3 1 4 0 4 1 4 1 2 3 2]


In [ ]:
def print_clusters(texts, labels, title):
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(texts[i])

    print(f"\n{title} Clusters:")
    for label, sentences in clusters.items():
        print(f"Cluster {label}:")
        for sentence in sentences:
            print(f"  {sentence}")

print_clusters(texts, cluster_labels, 'SciBERT')


SciBERT Clusters:
Cluster 1:
  function selectActivities(start, finish, N):
          activities.append((start[i], finish[i], i))
      sort(activities, by finish time)
      selectedActivities.append(activities[0])
              selectedActivities.append(activities[i])
      print "Number of activities selected:", length(selectedActivities)
Cluster 3:
      activities = []
      selectedActivities = []
      for each activity in selectedActivities:
Cluster 0:
      for i from 0 to N-1:
      for i from 1 to N-1:
Cluster 4:
      lastFinishTime = activities[0].finish
          if activities[i].start >= lastFinishTime:
              lastFinishTime = activities[i].finish
Cluster 2:
      print "Selected activities:"
          print activity.index


In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
valid_labels = cluster_labels != -1
if np.sum(valid_labels) > 1:  # Check if there are more than one valid sample
    silhouette_avg = silhouette_score(sentence_embeddings[valid_labels], cluster_labels[valid_labels])
    print(f"Silhouette Score: {silhouette_avg:.3f}")
else:
    print("Silhouette Score: Not applicable (less than 2 valid samples)")

Silhouette Score: 0.252
